In [7]:

# True: not answer all question，False: answer all questions mandatorily。
answer_all = True
# load Bert
model_checkpoint = "distilbert-base-uncased"

batch_size = 16


In [3]:
# qa pairs
import csv
with open('../testing_QA/reformatting_testing_qa.csv','r',encoding='utf-8') as f:
    reader = csv.DictReader(f)
    datasets = [row for row in reader]

# construct datasets
datasets 

[{'question': 'What was the purpose of the study on new-onset benign epilepsy with centrotemporal spikes (BECTS)?',
  'answer': 'T',
  'article_title': 'Neuropsychological profile in new-onset benign epilepsy with centrotemporal spikes (BECTS): Focusing on executive functions.',
  'PMID': '26667848',
  'author': "['Filippini M', 'Ardu E', 'Stefanelli S', 'Boni A', 'Gobbi G', 'Benso F']",
  'context': 'PURPOSE: Increased evidence of subnormal neuropsychological functioning in new-onset childhood epilepsy has been obtained, although results are still rare and controversial. With a prospective study, we aimed to define the very early neuropsychological profile of children with benign epilepsy with centrotemporal spikes (BECTS), including executive functions (EF) because of their key role in learning. Additionally, we enrolled drug-naive children, with a NREM sleep frequency of discharges <85% and with a Performance Intelligence Quotient equal or superior to 85, in order to exclude additio

In [14]:
from random import sample

training_data = sample(datasets, int(len(datasets)*0.8))
testing_data = [i for i in datasets if i not in training_data]

In [19]:
testing_data

[{'question': 'How many children with BECTS and healthy children were enrolled in the study?',
  'answer': 'F',
  'article_title': 'Neuropsychological profile in new-onset benign epilepsy with centrotemporal spikes (BECTS): Focusing on executive functions.',
  'PMID': '26667848',
  'author': "['Filippini M', 'Ardu E', 'Stefanelli S', 'Boni A', 'Gobbi G', 'Benso F']",
  'context': 'PURPOSE: Increased evidence of subnormal neuropsychological functioning in new-onset childhood epilepsy has been obtained, although results are still rare and controversial. With a prospective study, we aimed to define the very early neuropsychological profile of children with benign epilepsy with centrotemporal spikes (BECTS), including executive functions (EF) because of their key role in learning. Additionally, we enrolled drug-naive children, with a NREM sleep frequency of discharges <85% and with a Performance Intelligence Quotient equal or superior to 85, in order to exclude additional effects on the ne

In [4]:
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import torch
import collections
from tqdm.auto import tqdm

f:\Heidelberg\semester_5\INLPT\Final Project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
max_length = 384 # max length of input feature
doc_stride = 128 # repeat tokens number between 2 slices

In [10]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [11]:
tokenizer("What is your name?")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [20]:
for i, example in enumerate(training_data):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = training_data[i]

In [21]:
len(tokenizer(example["question"], example["context"])["input_ids"])

392

In [25]:
tokenized_example = tokenizer(
    example["question"], # question
    example["context"],  # abstract
    max_length=max_length, 
    truncation="only_second", # slice only in the context
    return_overflowing_tokens=True,  # flag the slicing if exceed max length
    return_offsets_mapping=True,
    stride=doc_stride # slice step size
)

In [26]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 162]

In [27]:
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 8), (9, 12), (13, 22), (23, 31), (32, 34), (35, 40), (41, 53), (54, 56), (57, 60), (61, 65), (65, 67), (67, 70), (71, 77), (78, 80), (81, 94), (95, 104), (105, 107), (108, 111), (112, 116), (116, 117), (118, 127), (127, 128), (0, 0), (0, 7), (7, 8), (9, 12), (13, 19), (20, 25), (26, 32), (33, 36), (37, 48), (49, 54), (55, 56), (56, 60), (60, 61), (61, 62), (63, 65), (66, 67), (68, 79), (79, 80), (81, 93), (94, 107), (108, 110), (111, 119), (119, 120), (121, 128), (129, 133), (134, 137), (138, 145), (146, 155), (156, 160), (161, 162), (162, 163), (163, 165), (166, 171), (172, 175), (176, 184), (185, 193), (193, 195), (195, 196), (197, 200), (201, 208), (209, 213), (213, 215), (215, 218), (219, 223), (224, 231), (232, 235), (236, 249), (249, 250), (251, 254), (255, 260), (261, 265), (266, 268), (269, 278), (278, 281), (282, 295), (296, 298), (298, 300), (300, 303), (303, 308), (309, 311), (312, 325), (325, 326), (327, 336), (336, 337), (338, 341), (342, 352), (353, 3

In [28]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 